In [ ]:
import pandas as pd

In [ ]:
import geocoder

In [ ]:
stp = geocoder.osm('Санкт-Петербург')

In [ ]:
stp.geojson

In [ ]:
import time
from tqdm import tqdm_notebook

In [ ]:
df = pd.read_csv('open_stp_data.csv')

In [ ]:
VO = df[df['addr_street'].str.contains('линия')]

In [ ]:
VO[VO['addr_building'].isna() != True]

In [ ]:
VI_LINES = {1: ('1-я линия В.О.',),
2: ('2-3-я линии В.О.',),
3: ('2-3-я линии В.О.',),
4: ('4-5-я линии В.О.',),
5: ('4-5-я линии В.О.',),
6: ('6-я линия В.О.', '6-7-я линии В.О.'),
7: ('7-я линия В.О.', '6-7-я линии В.О.'),
8: ('8-я линия В.О.',), 
9: ('9-я линия В.О.',),
10: ('10-11-я линии В.О.',),
11: ('10-11-я линии В.О.',),
12: ('12-13-я линии В.О.',),
13: ('12-13-я линии В.О.',),
14: ('14-я линия В.О.', '14-15-я линии В.О.'),
15: ('15-я линия В.О.', '14-15-я линии В.О.'),
16: ('16-я линия В.О.', '16-17-я линии В.О.'),
17: ('17-я линия В.О.', '16-17-я линии В.О.'),
18: ('18-19-я линии В.О.',),
19: ('18-19-я линии В.О.',),
20: ('20-21-я линии В.О.',),
21: ('20-21-я линии В.О.',),
22: ('22-23-я линии В.О.',),
23: ('22-23-я линии В.О.',),
24: ('24-25-я линии В.О.',),
25: ('24-25-я линии В.О.',),
26: ('26-27-я линии В.О.',),
27: ('26-27-я линии В.О.',)}

In [ ]:
VI_LINES

1
2-3
4-5
6-7 (6, 7)
8
9
10-11
12-13
14-15 (14, 15)
16-17 (16, 17)
18-19
20-21
22-23
24-25
26-27

In [ ]:
import re

In [ ]:
def loc_VO(string):
    try:
        string = re.sub(',', '', string)
        return VI_LINES[int(re.search('\d+', string).group(0))]
    except AttributeError:
        return (string.split(' ')[0] + ' линия В.О.', )

In [ ]:
loc_VO('16-я линия, В.О.')

In [ ]:
df['searchable'] = df.apply(lambda x: full_address(loc_VO(x.['locality'], str(x['addr_number']), str(x['addr_building'])), axis=1)

In [ ]:
def line_address(locality, number, building):
    if building == 'nan':
        return 'Санкт-Петербург, ' + locality + ' ' + number
    else:
        return 'Санкт-Петербург, ' + locality + ' ' + number + ' к' + building

In [ ]:
def standartize_address(street):
    
    shorthands = {'ул.': 'улица', 
                    'пр.': 'проспект',
                    'ш.': 'шоссе',
                    'пер.': 'переулок',
                    'бульв.': 'бульвар',
                    'наб.': 'набережная'}
    
    locality = None
    street = street.split(',')
    if len(street) == 2:
        locality = street[0]
        street = street[1]
    else:
        street = street[0]
    street = street.split(' ')
    if len(street) <= 1:
        return(street[0])
    else:
        status = street[-1]
        if status.lower() in shorthands:
            status = shorthands[status.lower()]
        if street[-2][-2:] in 'ая ий ый ое':
            if locality is not None:
                return ' '.join((locality + ',', ' '.join(street[:-1]), status))
            else:
                return ' '.join((' '.join(street[:-1]), status))
        else:
            if locality is not None:
                return ' '.join((locality + ',', status, ' '.join(street[0:-2]), street[-2]))
            else:
                return ' '.join((status, ' '.join(street[0:-2]), street[-2]))

In [ ]:
def full_address(locality, number, building):
    if building == 'nan':
        return 'Санкт-Петербург, ' + locality + ' ' + number
    else:
        return 'Санкт-Петербург, ' + locality + ' ' + number + ' к' + building

In [ ]:
for index, row in VO.iterrows():
    for locality in loc_VO(row['addr_street']):
        number = row['addr_number']
        building = row['addr_building']
        address = full_address(locality, str(number), str(building))

In [ ]:
import csv
with open('VI.csv', 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    for num, (index, row) in enumerate(VO.iterrows()):
        print(num)
        for locality in loc_VO(row['addr_street']):
            time.sleep(1)
            try:
                number = row['addr_number']
                building = row['addr_building']
                address = full_address(locality, str(number), str(building))
                g = geocoder.osm(address)
                if g.status == 'OK':
                    coords = (index, address, g.latlng[0], g.latlng[1])
                    print(coords)
                    writer.writerow(coords)
                    break
                else:
                    print()
            except Exception:
                print('ERROR', index, locality)
            

In [ ]:
df['locality'] = df['addr_street'].map(standartize_address)

In [ ]:
VO['searchable'] = VO.apply(lambda x: full_address(loc_VO(x['addr_street']), str(x['addr_number']), str(x['addr_building'])), axis=1)

In [ ]:
g = geocoder.osm('Санкт-Петербургб фыва')
g.status

In [ ]:
import csv
with open('coordinates2.csv', 'a') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                        quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for address in tqdm_notebook(zip(df['searchable'][4372:], df.index[4372:])):
        try:
            g = geocoder.osm(address[0])
            if g.status == 'OK':
                coords = (address[1], g.latlng[0], g.latlng[1])
            else:
                coords = (address[1], None, None)
            time.sleep(1)
            writer.writerow(coords)
        except Exception:
            pass

In [ ]:
for i in zip(df['searchable'], df.index):
    print(i)

In [ ]:
df['searchable'][0]

In [ ]:
a = [None,
 [59.8371606, 30.2628121],
 [59.86311185, 30.3760026393255],
 [59.8642619, 30.3898461760562],
 [59.7035591, 30.3608674008901],
 None,
 [59.7504734, 30.5888801267241],
 None,
 [59.9036119, 29.7696169],
 [60.0249241, 30.3817525],
 [59.7423442, 30.58989755],
 [59.9275836, 30.4125903],
 [59.8528132, 30.3114165],
 [59.8796156, 30.4337016104022],
 [59.8908475, 30.4281587],
 [59.99629555, 29.7642317527372],
 [59.74334865, 30.6046086],
 None,
 [59.91176595, 30.349100302754],
 [59.9179702, 30.3499406696604],
 None,
 [59.717854, 30.4060592],
 [59.9362189, 30.3676803],
 [59.9296829, 30.3507480065549],
 [59.91274655, 30.3508399965845],
 [59.9517114, 30.410437],
 [60.0063945, 30.4066724878941],
 [59.92449115, 30.3493128634555],
 [59.8994886, 30.3682884567568],
 [59.9001299, 30.3619947],
 [59.9441589, 30.3503358],
 None,
 None,
 [59.95670085, 30.411813],
 [59.91485055, 30.2989521073385],
 [59.8353306, 30.3917037365553],
 [59.74962175, 30.6065479785519],
 [59.83501975, 30.392958973801],
 [59.74819685, 30.6135747863541],
 [59.9403751, 30.4962290355362],
 [59.947037, 30.3823762],
 [60.0243176, 30.2416869986099],
 [59.9391854, 30.4793239],
 [59.9485886, 30.4812072539105],
 [59.95840535, 30.4104488127528],
 [59.9131809, 30.2954147],
 [59.9263417, 30.3053269963056],
 [59.933532, 30.2393263],
 [59.83765525, 30.204220478898],
 [59.9169766, 30.305427],
 None,
 None,
 [59.86013445, 30.24498485],
 [59.8579924, 30.2395216],
 [59.99752585, 29.7632663023832],
 None,
 None,
 [59.9022387, 30.2707897],
 [60.00525245, 30.2458204688005],
 [60.00112945, 30.2438787849814],
 [59.86697065, 30.4375364597524],
 None,
 [59.7337312, 30.07563345],
 [59.98359715, 30.3113735429746],
 [59.9828169, 30.3105342],
 [59.99523715, 30.314605331149],
 [59.8400397, 30.1766449316235],
 [59.9559682, 30.3051290128978],
 None,
 [60.02436625, 30.2920052],
 [59.8748216, 30.3011829643372],
 [59.88068195, 30.4723697394493],
 [59.8601483, 30.3268943703252],
 [60.0203177, 30.321585],
 None,
 [59.9893047, 30.3147250048941],
 [59.87311005, 30.4774082122105],
 [59.8700443, 29.8444771],
 [59.9245653, 30.3173009],
 None,
 [59.72980965, 30.4210766180108],
 [59.930162, 30.3499601],
 [59.91785965, 30.2929938033821],
 [59.9016395, 30.4252848],
 [59.95189345, 30.4145691448982],
 [59.83355765, 30.1516957749324],
 [60.03231565, 30.419415651008],
 [59.9952346, 29.7645067],
 [59.9922424, 29.7611233327738],
 [59.72685445, 30.4224172],
 [59.87694995, 30.2765652715838],
 [59.98570475, 30.2791587164922],
 [59.9951923, 29.7605425],
 [59.92523745, 30.4225420217657],
 [59.74829335, 30.5639815054703],
 [59.900647, 30.3697858],
 [59.788558, 30.1472063],
 [59.95053115, 30.4162297697019],
 [60.00117115, 29.7570686704303],
 [59.9767734, 30.4208794967604]]

In [ ]:
! pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
aa = [elem  for elem in a if elem is not None]

In [ ]:
x = [a[0] for a in aa]
y = [a[1] for a in aa]

In [ ]:
plt.plot(x, y, lw=0, marker='o')

In [ ]:
! pip install tqdm